In [1]:
%matplotlib ipympl 
#matplotlib interactive for IPython

#import libraries and functions
import pandas as pd
import math
import serial
import ipywidgets, asyncio, time, ipydatetime
from datetime import datetime
import matplotlib.pyplot as plt # to make mpl faster
import matplotlib as mpl # to make mpl faster
import numpy as np #numpy for tons of stuff
import Aeth51 as ae #main python file to use functions from there

#styles to limit the description width
numberStyle = {'description_width': "12%"}
numberstyle2  = {'description_width': "40%"}
itemLayout = ipywidgets.Layout(width = "auto",height = "30px")

def colLayout(wid):
    return ipywidgets.Layout(display = "flex",flex_flow='column',border = '1px solid black',align_items = "stretch",justify_content="flex-start",width = wid)

#create widgets
running = ipywidgets.ToggleButton(description="start", icon="play",button_style = 'danger', layout = itemLayout) #whether program is running or not
aethCOM = ipywidgets.Textarea(description = "COM #", layout = itemLayout, value = "COM6")
baudrate = ipywidgets.IntText(description = "Baud" , layout = itemLayout,value = "500000")
basic = ipywidgets.VBox(children = [running,aethCOM,baudrate],layout = colLayout("20%"))

timeBase = ipywidgets.IntText(description = "Timebase", value = 1,layout = itemLayout) #AE51 timebase
date = ipywidgets.Textarea(description = "Date", layout = itemLayout)
currentTime = ipywidgets.Textarea(description = "Time", layout = itemLayout)
timeBox = ipywidgets.VBox(children = [timeBase,date,currentTime],layout = colLayout("20%"))

power = ipywidgets.Textarea(description = "Power",layout = itemLayout)
streaming = ipywidgets.Textarea(description = "Streaming",layout = itemLayout)
shutdown = ipywidgets.Textarea(description = "Shutdown",layout = itemLayout)
powerBox = ipywidgets.VBox(children = [power, streaming,  shutdown], layout = colLayout("20%"))

battery = ipywidgets.Textarea(description = "Battery",layout = itemLayout)
flow = ipywidgets.Textarea(description = "Flow mlpm",layout = itemLayout)
PCBTemp = ipywidgets.Textarea(description = "PCB Temp",layout = itemLayout)
opBox = ipywidgets.VBox(children = [battery, flow,  PCBTemp], layout = colLayout("20%"))
#create widget layout

reference = ipywidgets.Textarea(description = "Reference",layout = itemLayout)
sensor = ipywidgets.Textarea(description = "Sensor1",layout = itemLayout)
feedback = ipywidgets.Textarea(description = "Feedback",layout = itemLayout)
dataBox = ipywidgets.VBox(children = [reference, sensor, feedback], layout = colLayout("20%"))

sound = ipywidgets.Textarea(description = "Sound notif.",layout = itemLayout)
status = ipywidgets.Textarea(description = "status",layout = itemLayout)
sessions = ipywidgets.IntText(description = "sessions" , layout = itemLayout)
statusBox = ipywidgets.VBox(children = [status, sound,sessions], layout = colLayout("20%"))

switch = ipywidgets.RadioButtons(description = "Mode",options = ["get","set"],value = "get",layout={'width': "150px"})

chooseTimebase = ipywidgets.IntText(description = 'Set Timebase', value = 60,layout = {'width':'220px'})
timeBaseButton = ipywidgets.Button(description = 'timebase',icon='fa-database')
setTimeBaseBox = ipywidgets.HBox([chooseTimebase,timeBaseButton])

chooseHour = ipywidgets.IntText(description='Set Time',layout = {'width':'120px'})
chooseMinute = ipywidgets.IntText(description=':',layout = {'width':'50px'},style = numberStyle)
chooseSecond = ipywidgets.IntText(description=':',layout = {'width':'50px'},style = numberStyle)
timeButton = ipywidgets.Button(description = 'time',icon='fa-clock-o')
setTimeBox = ipywidgets.HBox([chooseHour,chooseMinute,chooseSecond,timeButton])

chooseDate = ipywidgets.DatePicker(description='Set Date',layout = {'width':'220px'})
dateButton = ipywidgets.Button(description = 'date',icon='fa-calendar')
setDateBox = ipywidgets.HBox([chooseDate,dateButton])

setDateTime = ipywidgets.VBox([setTimeBaseBox,setDateBox,setTimeBox],layout = {'width':'300px'})

flashButton = ipywidgets.Button(description = 'flash', icon = 'fa-bolt',layout = {'width':'97%'})
streamButton = ipywidgets.Button(description = 'streaming', icon = 'fa-wifi',layout = {'width':'97%'})
shutdownMode = ipywidgets.Dropdown(decription = 'shutdown mode',options = ['Simple','Normal','Disabled'],layout = {'width':'48%'})
shutdownButton = ipywidgets.Button(description = 'shutdown mode', icon = 'fa-chevron-circle-down',layout = {'width':'48%'})
shutdownBox = ipywidgets.HBox([shutdownMode,shutdownButton])
modeBox = ipywidgets.VBox([flashButton,streamButton,shutdownBox],layout = {'width':'250px'})

soundMode = ipywidgets.Dropdown(decription = 'sound mode',options = ['on','off'],layout = {'width':'48%'})
soundButton = ipywidgets.Button(description = 'sound notifs', icon = 'fa-volume-up',layout = {'width':'48%'})
soundBox = ipywidgets.HBox([soundMode,soundButton],layout = {'width':'96%'})
kill = ipywidgets.Button(description = "kill", icon ="fa-power-off",layout = {'width':'96%'})
lastBox = ipywidgets.VBox([soundBox,kill],layout = {'width':'250px'})

sessionButton = ipywidgets.Button(description = 'get session #', icon = 'fa-slack',layout = {'width':'96%'})
batteryButton = ipywidgets.Button(description = 'get battery %', icon = 'fa-battery-three-quarters',layout = {'width':'96%'})
flowButton = ipywidgets.Button(description = 'get flow', icon = 'fa-arrow-circle-right',layout = {'width':'96%'})

Displays = ipywidgets.HBox([basic,timeBox,powerBox,opBox,dataBox,statusBox],layout=ipywidgets.Layout(width = "96%"))
setButtons = ipywidgets.HBox([switch,setDateTime,modeBox,lastBox],layout=ipywidgets.Layout(width = "96%", height = "20%",padding = '10px'))
Console = ipywidgets.Textarea(description = "console",layout=ipywidgets.Layout(width = "98%",height = "60%"))

ui = ipywidgets.VBox([Displays,setButtons,Console],layout=ipywidgets.Layout(width = "80%",height = "800px"))

streams  = []

def changeDate(change):
    if switch.value == 'set':
        year = (int(chooseDate.value.year) - 2000).to_bytes(1,'big')
        month = int(chooseDate.value.month).to_bytes(1,'big')
        day = int(chooseDate.value.day).to_bytes(1,'big')
        dayOfWeek = datetime(chooseDate.value.year,chooseDate.value.month,chooseDate.value.day).weekday().to_bytes(1,'big')
        ae.write(b'AE5X:D' + year + month + day + dayOfWeek)
        updateConsole(str(chooseDate.value.month) + '/' + str(chooseDate.value.day) + '/' + str(chooseDate.value.year))
        date.value = str(chooseDate.value.month) + '/' + str(chooseDate.value.day) + '/' + str(chooseDate.value.year)
    else:
        getAeth(b'AE5X:D',date,3,"Date",ae.hexDate) #date
        
def changeTimeBase(change):
    if switch.value == 'set':
        new = chooseTimebase.value.to_bytes(2,'little')
        updateConsole(str(b'AE5X:X'+new))
        ae.write(b'AE5X:X'+new)
    else:
        getAeth(b'AE5X:X',timeBase,2,"Timebase",ae.hexFlow2) #timebase

def changeTime(change):
    if switch.value == 'set':
        hour = chooseHour.value.to_bytes(1,'big')
        minute = chooseMinute.value.to_bytes(1,'big')
        second = chooseSecond.value.to_bytes(1,'big')
        ae.write(b'AE5X:T' + hour + minute + second)
        updateConsole(str(b'AE5X:T' + hour + minute + second))
    else:
        getAeth(b'AE5X:T',currentTime,3,"Time",ae.hexTime) #time
    
def changeFlash(change):
    if switch.value == 'set':
        updateConsole(str(b'AE5X:M\x46'))
        ae.write(b'AE5X:M\x46')
    else:
        getAeth(b'AE5X:M',streaming,1,"Streaming mode",ae.hexStreaming)

def changeStream(change):
    if switch.value == 'set':
        updateConsole(str(b'AE5X:M\x42'))
        ae.write(b'AE5X:M\x42')
    else:
        getAeth(b'AE5X:M',streaming,1,"Streaming mode",ae.hexStreaming)

def changeShutdown(change):
    if switch.value == 'set':
        if shutdownMode.value == 'Simple':
            char = b'S'
        if shutdownMode.value == 'Normal':
            char = b'N'
        if shutdownMode.value == 'Disabled':
            char = b'U'
        ae.write(b'AE5X:P' + char)
        updateConsole(str(b'AE5X:P' + char))
    else:
        getAeth(b'AE5X:P',shutdown,1,"Shutdown mode",ae.hexShutdown) 

def changeSound(change):
    if switch.value == 'set':
        if soundMode.value == 'on': 
            char = '\x64'
        if soundMode.value == 'off':
            char = '\x00'
        ae.write(b'AE5X:U' + char)
        updateConsole(str(b'AE5X:U' + char))
    else:
        getAeth(b'AE5X:U',sound,1,"Sound notifications",ae.hexSound) 

def killNow(change):
    if switch.value == 'set':
        updateConsole('killing')
        updateConsole(str(b'AE5X:K'))
        ae.write(b'AE5X:K')

def getFlow(change):
    getAeth(b'AE5X:F',flow,1,"Flow",ae.hexFlow)

def getSession(change):
    getAeth(b'AE5X:N',sessions,2,"Sessions ",ae.hexSessions) 

def getBattery(change):
    getAeth(b'AE5X:B',battery,2,"Battery",ae.hexBattery) 

#for managing asyncronous ts=asks
tasks = dict()
    
#update all matplotlib plots and get instrument data
def switchWidgets():
    if switch.value == 'get':
        setDateTime.children = [timeBaseButton,dateButton,timeButton]
        modeBox.children = [streamButton,shutdownButton,sessionButton]
        lastBox.children = [batteryButton,flowButton,sessionButton]
        timeBaseButton.layout.width = "96%"
        dateButton.layout.width = "96%"
        timeButton.layout.width = "96%"
        shutdownButton.layout.width = "96%"
        sessionButton.layout.width = "96%"
    elif switch.value == 'set':
        setDateTime.children = [setTimeBaseBox,setDateBox,setTimeBox]
        modeBox.children = [flashButton,streamButton,shutdownBox]
        lastBox.children = [soundBox,kill]
        timeBaseButton.layout.width = "48%"
        dateButton.layout.width = "48%"
        timeButton.layout.width = "48%"
        shutdownButton.layout.width = "48%"
        sessionButton.layout.width = "48%"

def getStream():
    stream = ae.getResponse()
    print('stream',stream)
    if stream != None:
        updateValues(stream)
            
def updateValues(stream):
    if b"AE5X:M" in stream and len(stream)>30:
        start = stream.index(b"AE5X:M")
        stream = stream[start-2:]
        streamDict = ae.streamToDict(stream)
        updateConsole(streamDict)
        reference.value = str(streamDict["reference"])
        sensor.value = str(streamDict['sensor 1'])
        feedback.value = str(streamDict['feedback'])
        flow.value = str(streamDict['flow'])
        PCBTemp.value = streamDict['PCB temp']
        date.value = streamDict['date']
        time.value = streamDict['time']
        status.value = streamDict['status']
        battery.value = streamDict['battery']

def updateConsole(string):
    Console.value =  str(string) + '\n\n' + Console.value

def getReply(message):
    ae.write(message)
    updateConsole("writing: " + str(message))
    reply = ae.getResponse()
    updateConsole(str(reply))
    start = reply.index(message)
    if start != 2:
        extra = reply[0:start-2]
        updateValues(extra)
        
    reply = reply[start-2:]
    if reply[2:8] == message:
        return reply
        
def getAeth(message, widget, byteCount, varName, filterFunction):
    reply = getReply(message)
    try:
        widget.value = filterFunction(reply[8:8+byteCount])
        updateConsole(varName + ": " + str(widget.value))
    except Exception as e:
        print(e)
    
#run main loop of asyncronous coroutines
async def do_loop():
    t=0
    getAeth(b'AE5X:X',timeBase,2,"Timebase",ae.hexFlow2) #timebase
    getAeth(b'AE5X:D',date,3,"Date",ae.hexDate) #date
    getAeth(b'AE5X:T',currentTime,3,"Time",ae.hexTime) #time
    getAeth(b'AE5X:O',power,1,"Power",ae.hexPower) #Power
    getAeth(b'AE5X:M',streaming,1,"Streaming mode",ae.hexStreaming) 
    getAeth(b'AE5X:P',shutdown,1,"Shutdown mode",ae.hexShutdown) 
    getAeth(b'AE5X:B',battery,2,"Battery",ae.hexBattery) 
    getAeth(b'AE5X:F',flow,1,"Flow",ae.hexFlow)
    getAeth(b'AE5X:U',sound,1,"Sound notifications",ae.hexSound) 
    getAeth(b'AE5X:N',sessions,2,"Sessions ",ae.hexSessions) 

    while running.value:
        switchWidgets()
        getStream()
        t += 1
        await asyncio.sleep(1.2)

#stop or start main loop according to the start/stop button
def on_running_changed(*change):
    task = tasks.pop("do_loop", None)
   
    if task:
        task.cancel()
        ae.closePort()
        updateConsole("port closed")
        updateConsole("________________________________")
    if running.value:
        updateConsole("starting...")
        try: 
            ae.openAethPort(aethCOM.value,baudrate.value)
            updateConsole("COM port open!")
        except:
            updateConsole("Port access denied!")
        ae.flush()
        time.sleep(0.1)
        tasks["do_loop"] = asyncio.get_event_loop().create_task(do_loop())

timeBaseButton.on_click(changeTimeBase)
timeButton.on_click(changeTime)
dateButton.on_click(changeDate)
flashButton.on_click(changeFlash)
streamButton.on_click(changeStream)
shutdownButton.on_click(changeShutdown)
soundButton.on_click(changeSound)
kill.on_click(killNow)
sessionButton.on_click(getSession)
flowButton.on_click(getFlow)
batteryButton.on_click(getBattery)

#observe stop/start button
running.observe(on_running_changed, "value")

#display UI (widget grid)
ui

In [7]:
streamDict = {'reference': 812311, 'sensor 1': 916916, 'feedback': 801689, 'flow': 99, 'PCB temp': '42C', 'date': '8/15/2022', 'time': '04:08:00', 'status': '0', 'battery': '100%', 'CRC': 144}
reference.value = str(streamDict["reference"])

In [3]:
str({'reference': 812309, 'sensor 1': 917114, 'feedback': 801688, 'flow': 99, 'PCB temp': '42C', 'date': '8/15/2022', 'time': '04:03:00', 'status': '0', 'battery': '100%', 'CRC': 85}
   )

"{'reference': 812309, 'sensor 1': 917114, 'feedback': 801688, 'flow': 99, 'PCB temp': '42C', 'date': '8/15/2022', 'time': '04:03:00', 'status': '0', 'battery': '100%', 'CRC': 85}"